In [1]:
!pip install keras cython h5py --quiet

In [2]:
import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

Using TensorFlow backend.


## Explore the Data

References: https://www.cs.toronto.edu/~kriz/cifar.html

The CIFAR-10 is the cononical dataset for training image classfier models in machine learning.

The dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. 


The encoded targets are as follows:
    
       0 -- airplane
       1 -- automobile
       2 -- bird
       3 -- cat
       4 -- deer 
       5 -- dog
       6 -- frog
       7 -- horse
       8 -- ship
       9 -- truck

In [3]:
from keras.datasets import cifar10

In [4]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [ ]:
# # print 'target: ', y_train[1:36]
# fig = plt.figure(1100)
# sample = np.random.random_integers(1,6000,10)
# for i in range(1,len(sample)):

#     fig.add_subplot(330+i)
#     plt.imshow(X_train[sample[i]], cmap='Greys')

In [ ]:
# print('Dimension of the training data is: {}'.format(X_train.shape))

Exploring the RGB channels of the images

In [ ]:
y_train[1]

In [ ]:
# blue channel
# this_imageimage = X_train[1][:][:][:]
image = X_train[1][:][:][:]

this_image = image.astype(np.float64)
this_image

plt.imshow(this_image)

# red
this_image[:,:,0]

zeroes = np.zeros_like(this_image[:,:,0])

plt.imshow(np.dstack([this_image[:,:,0],zeroes,zeroes]))
plt.show()
plt.imshow(np.dstack([zeroes,this_image[:,:,1],zeroes]))
plt.show()
plt.imshow(np.dstack([zeroes,zeroes,this_image[:,:,2]]))
plt.show()

this_image[:,:,1].astype(np.float64)

this_image[:,:,2].astype(np.float64)

## Preprocess the data

Normalize the data. Pixel values of images are bwtween 0 and 255, and we'd like them to be floats between 0 and 1.

In [5]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

MemoryError: 

We need to one-hot encode our Ys. Luckily, Keras has a utility to do this for us!

In [ ]:
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [ ]:
X_train.shape

## Multilayer perceptron model

Before we get into CNNs, we will first build a multi-layer perceptron (MLP). This involves an input layer, a layer to flatten the data, and an output layer.

To build it, we will use the **sequential model**. Here, we can **add** layers in a sequential list, **compile** the model with an optimizer, loss function, and scoring metric, then **fit** the model to the data. 

In [ ]:
from keras.layers import Input, Conv2D, MaxPool2D, Dropout, Activation, Dense, Flatten
from keras.models import Sequential
from keras.activations import relu, softmax
from keras import layers as l

In [ ]:
mlp = Sequential([
    Dense(64, input_shape=(32,32,3), activation='relu', name='input'),
    Flatten(name = 'flatten'),
    Dense(10, activation='softmax', name = 'output')
])

In [ ]:
# mlp = Sequential([
#     l.InputLayer(input_shape=(32,32,3), name='input'),
#     l.Flatten(),
#     Dense(24, activation='relu'),
#     Dense(10, activation='softmax', name='output')
# ])

In [ ]:
mlp.summary()

In [ ]:
mlp.compile(optimizer=keras.optimizers.SGD(),
           loss=keras.losses.categorical_crossentropy,
           metrics=['accuracy'])

In [ ]:
mlp.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)

In [ ]:
loss, accuracy = mlp.evaluate(X_test, y_test)

In [ ]:
print('MLP Loss: {}'.format(loss))
print('MLP Accuracy: {}'.format(accuracy))

## The Convolution Neural Network

In [ ]:
from keras.layers import Input, Conv2D, MaxPool2D, Dropout, Activation, Dense, Flatten
from keras.models import Sequential
from keras.activations import relu, softmax
from keras.layers.advanced_activations import LeakyReLU
from keras import layers as l

In [ ]:
# cnn = keras.models.load_model('cnn.h5')

In [ ]:
# Model 1
# cnn = Sequential([
#     Conv2D(2, kernel_size=(3, 3),
#                  activation='relu',
#                  input_shape=(32,32,3)),
#     Conv2D(16, (3, 3), activation='relu'),
#     Dropout(.25),
#     MaxPool2D(),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dropout(.25),
#     Dense(10, activation='softmax')
# ])

In [ ]:
# Model 2
# cnn = Sequential([
#     Conv2D(2, kernel_size=(3, 3),
#                  activation='relu',
#                  input_shape=(32,32,3)),
#     MaxPool2D(pool_size=(2,2)),
#     Conv2D(2, kernel_size=(3, 3), activation='relu'),
#     MaxPool2D(),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dense(10, activation='softmax')
# ])

In [ ]:
# Model 3
cnn = Sequential([
    Conv2D(16, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(32,32,3),padding='same' ),
    Conv2D(16, kernel_size=(3, 3),
                 activation=None,padding='same' ),
    MaxPool2D(pool_size=(2,2)),
    Conv2D(8, kernel_size=(3, 3),
                 activation='relu',padding='same'),
    Conv2D(8, kernel_size=(3, 3),
                 activation=None,padding='same'),
    MaxPool2D(pool_size=(2,2)),
    Conv2D(4, kernel_size=(3, 3),
                 activation='relu',padding='same'),
    Conv2D(4, kernel_size=(3, 3),
                 activation=None,padding='same'),
    MaxPool2D(pool_size=(2,2)),
    Conv2D(2, kernel_size=(3, 3),
                 activation='relu',padding='same'),
    Conv2D(2, kernel_size=(3, 3),
                 activation=None,padding='same'),
    MaxPool2D(pool_size=(2,2)),
    Conv2D(2, kernel_size=(3, 3),
                 activation='relu',padding='same'),
    Conv2D(2, kernel_size=(3, 3),
                 activation=None,padding='same'),
    MaxPool2D(pool_size=(2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

In [ ]:
cnn.summary()

In [ ]:
cnn.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
cnn.fit(X_train, y_train, validation_split=.3, epochs=55)

In [ ]:
cnn.save('cnn_model3_epoch60.h5')

In [ ]:
loss, accuracy = cnn.evaluate(X_test, y_test)

In [ ]:
print('CNN Loss: {}'.format(loss))
print('CNN Accuracy: {}'.format(accuracy))

In [ ]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

In [ ]:
# pwd
# curr_dir = !pwd
# path = curr_dir[0]+'/Graph'

In [ ]:
# RUn this to implement tensorboard callbacks
cnn.fit(X_train, y_train, validation_split=.3, epochs=10,callbacks=[tbCallBack])

In [ ]:
y_pred_probas = cnn.predict(X_test)

In [ ]:
y_pred = np.argmax(y_pred_probas, axis=1)

In [ ]:
y_pred

In [ ]:
np.argmax(y_test, axis=1)

SAVE and LOAD the models

In [ ]:
#cnn.save('cnn.h5')

In [ ]:
#mlp.save('mlp.h5')

In [ ]:
# keras.models.load_model('cnn.h5')